# Multi-Head Attention

Multi-Head Attention is an important part of all Transformer-based models.
This tutorial will show how to write it and how to then optimize it.

In [1]:
import torch
import ttnn

torch.manual_seed(0)

device_id = 0
device = ttnn.open(device_id)

                  Metal | INFO     | Initializing device 0
                 Device | INFO     | Opening device driver
2023-11-24 20:47:26.648 | INFO     | SiliconDriver   - Detected 1 PCI device
2023-11-24 20:47:26.658 | WARNING  | SiliconDriver   - init_detect_tt_device_numanodes(): Could not determine NumaNodeSet for TT device (physical_device_id: 0 pci_bus_id: 0000:00:08.0)
2023-11-24 20:47:26.658 | WARNING  | SiliconDriver   - Could not find NumaNodeSet for TT Device (physical_device_id: 0 pci_bus_id: 0000:00:08.0)
2023-11-24 20:47:26.658 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 0 device_id: 0xfaca revision: 0)
2023-11-24 20:47:26.658 | WARNING  | SiliconDriver   - bind_area_memory_nodeset(): Unable to determine TT Device to NumaNode mapping for physical_device_id: 0. Skipping membind.
---- ttSiliconDevice::init_hugepage: bind_area_to_memory_nodeset() failed (physical_device_id: 0 ch: 0). Hugepage allocation is not on NumaN

## Write Multi-Head Attention using ttnn

In [2]:
def multi_head_attention(
    hidden_states,
    query_weight,
    query_bias,
    key_weight,
    key_bias,
    value_weight,
    value_bias,
    output_weight,
    output_bias,
    *,
    head_size,
):
    batch_size, sequence_size, hidden_size = hidden_states.shape
    num_heads = hidden_size // head_size

    query = hidden_states @ query_weight
    query = query + query_bias
    query = ttnn.reshape(query, (batch_size, sequence_size, num_heads, head_size))
    query = ttnn.permute(query, (0, 2, 1, 3))

    key = hidden_states @ key_weight
    key = key + key_bias
    key = ttnn.reshape(key, (batch_size, sequence_size, num_heads, head_size))
    key = ttnn.permute(key, (0, 2, 3, 1))

    value = hidden_states @ value_weight
    value = value + value_bias
    value = ttnn.reshape(value, (batch_size, sequence_size, num_heads, head_size))
    value = ttnn.permute(value, (0, 2, 1, 3))

    attention_scores = query @ key
    attention_scores = attention_scores * (1 / (head_size**0.5))
    attention_probs = ttnn.softmax(attention_scores, dim=-1)

    context_layer = attention_probs @ value
    context_layer = ttnn.permute(context_layer, (0, 2, 1, 3))
    context_layer = ttnn.reshape(context_layer, (batch_size, sequence_size, hidden_size))

    self_output = context_layer @ output_weight
    self_output = self_output + output_bias

    return self_output

Now that the model is written, let's create input tensors to run it and test it

## Configuration

In [3]:
batch_size = 8
sequence_size = 384
num_heads = 16
head_size = 64
hidden_size = num_heads * head_size

## Initialize activations and weights using torch

In [4]:
torch_hidden_states = torch.randn((batch_size, sequence_size, hidden_size), dtype=torch.bfloat16)
torch_query_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_query_bias = torch.randn((hidden_size,), dtype=torch.bfloat16)
torch_key_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_key_bias = torch.randn((hidden_size,), dtype=torch.bfloat16)
torch_value_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_value_bias = torch.randn((hidden_size,), dtype=torch.bfloat16)
torch_output_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_output_bias = torch.randn((hidden_size,), dtype=torch.bfloat16)

## Convert activations and weights to ttnn

In [5]:
hidden_states = ttnn.from_torch(torch_hidden_states)
query_weight = ttnn.from_torch(torch_query_weight)
query_bias = ttnn.from_torch(torch_query_bias)
key_weight = ttnn.from_torch(torch_key_weight)
key_bias = ttnn.from_torch(torch_key_bias)
value_weight = ttnn.from_torch(torch_value_weight)
value_bias = ttnn.from_torch(torch_value_bias)
output_weight = ttnn.from_torch(torch_output_weight)
output_bias = ttnn.from_torch(torch_output_bias)

## Move activations and weights to device

In [6]:
hidden_states = ttnn.to_device(hidden_states, device)
query_weight = ttnn.to_device(query_weight, device)
query_bias = ttnn.to_device(query_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)
key_weight = ttnn.to_device(key_weight, device)
key_bias = ttnn.to_device(key_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)
value_weight = ttnn.to_device(value_weight, device)
value_bias = ttnn.to_device(value_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)
output_weight = ttnn.to_device(output_weight, device)
output_bias = ttnn.to_device(output_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)

## Run Multi-Head Attention

In [7]:
output = multi_head_attention(
    hidden_states,
    query_weight,
    query_bias,
    key_weight,
    key_bias,
    value_weight,
    value_bias,
    output_weight,
    output_bias,
    head_size=head_size,
)

                     Op | INFO     | Program of Operation tt::tt_metal::Tilize                               finished in     0.462033387 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::Tilize                               finished in     0.282176691 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::Matmul                               finished in     0.502522449 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::TilizeWithValPadding                 finished in      0.51778303 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::EltwiseBinaryBroadcast               finished in     0.477616286 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::Untilize                             finished in      0.47113988 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::TilizeWithValPadding                 finished in     0.585

## Explore the output

In [8]:
print("Printing 32 values of the first row of the output")
output = ttnn.to_layout(output, ttnn.ROW_MAJOR_LAYOUT)
output = ttnn.from_device(output)
print(output[0, :1, :32])

Printing 32 values of the first row of the output
Tensor([ [-0.3125, -0.212891, 0.0517578, -1.3125, -0.186523, 0.542969, -0.308594, 0.617188, 0.808594, -0.0991211, 0.726562, 0.773438, 0.390625, -0.233398, 1.25, -2.35938, 0.129883, -0.9375, 2.89062, -0.894531, 1.09375, 1.03906, 0.925781, -1.03906, -0.792969, -0.898438, 0.0708008, -1.55469, -2.29688, -1.33594, 0.652344, -1.36719]], dtype=bfloat16 )

                     Op | INFO     | Program of Operation tt::tt_metal::Untilize                             finished in     0.006195218 seconds


## Write optimized version of Multi-Head Attention

In [9]:
def optimized_multi_head_attention(
    hidden_states,
    fused_qkv_weight,
    fused_qkv_bias,
    self_output_weight,
    self_output_bias,
    *,
    head_size,
    num_cores_x=12,
):
    batch_size, *_ = hidden_states.shape

    fused_qkv_output = ttnn.matmul(
        hidden_states,
        fused_qkv_weight,
        bias=fused_qkv_bias,
        memory_config=ttnn.L1_MEMORY_CONFIG,
        dtype=ttnn.bfloat8_b,
        core_grid=(batch_size, num_cores_x),
    )

    (
        query,
        key,
        value,
    ) = ttnn.nlp.split_fused_qkv_and_split_heads(
        fused_qkv_output,
        memory_config=ttnn.L1_MEMORY_CONFIG,
        core_grid=(batch_size, num_cores_x),
    )
    ttnn.free(fused_qkv_output)

    attention_scores = ttnn.matmul(
        query,
        key,
        memory_config=ttnn.L1_MEMORY_CONFIG,
        dtype=ttnn.bfloat16,
        core_grid=(batch_size, num_cores_x),
    )
    ttnn.free(query)
    ttnn.free(key)

    attention_probs = ttnn.nlp.attention_softmax(attention_scores, attention_mask=None, head_size=head_size)

    context_layer = ttnn.matmul(
        attention_probs,
        value,
        memory_config=ttnn.L1_MEMORY_CONFIG,
        dtype=ttnn.bfloat8_b,
        core_grid=(batch_size, num_cores_x),
    )
    ttnn.free(attention_probs)

    context_layer = ttnn.nlp.concatenate_heads(
        context_layer,
        memory_config=ttnn.L1_MEMORY_CONFIG,
        core_grid=(batch_size, num_cores_x),
    )

    self_output = ttnn.matmul(
        context_layer,
        self_output_weight,
        bias=self_output_bias,
        memory_config=ttnn.L1_MEMORY_CONFIG,
        dtype=ttnn.bfloat16,
        core_grid=(batch_size, num_cores_x),
    )
    ttnn.free(context_layer)

    return self_output

## Pre-process the parameters of the optimized model

1. Fuse QKV weights and biases
2. Reshape and tilize for the optimized operations using preprocess_linear_weight and preprocess_linear_bias
3. Move to device

In [10]:
from ttnn.model_preprocessing import (
    ParametersConfig,
    preprocess_linear_bias,
    preprocess_linear_weight,
)

torch_qkv_weight = torch.cat([torch_query_weight, torch_key_weight, torch_value_weight], dim=-1)
torch_qkv_bias = torch.cat([torch_query_bias, torch_key_bias, torch_value_bias], dim=-1)

parameters_config = ParametersConfig()
qkv_weight = preprocess_linear_weight(parameters_config, torch_qkv_weight.T)
qkv_bias = preprocess_linear_bias(parameters_config, torch_qkv_bias)
output_weight = preprocess_linear_weight(parameters_config, torch_output_weight.T)
output_bias = preprocess_linear_bias(parameters_config, torch_output_bias)

qkv_weight = ttnn.to_device(qkv_weight, device)
qkv_bias = ttnn.to_device(qkv_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)
output_weight = ttnn.to_device(output_weight, device)
output_bias = ttnn.to_device(output_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)

## Run optimized model

In [11]:
hidden_states = ttnn.to_layout(hidden_states, ttnn.TILE_LAYOUT)
optimized_output = optimized_multi_head_attention(
    hidden_states,
    qkv_weight,
    qkv_bias,
    output_weight,
    output_bias,
    head_size=head_size,
)

                     Op | INFO     | Program of Operation tt::tt_metal::Tilize                               finished in     0.005049555 seconds
                     Op | INFO     | Program of Operation tt::operations::primary::Matmul                    finished in     0.640819076 seconds
                     Op | INFO     | Program of Operation tt::operations::primary::transformers::SplitFusedQKVAndSplitHeads finished in     0.460269814 seconds
                     Op | INFO     | Program of Operation tt::operations::primary::Matmul                    finished in     0.616765809 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::EltwiseBinaryBroadcast               finished in     0.390438683 seconds
                     Op | INFO     | Program of Operation tt::operations::primary::Softmax                   finished in     0.002576436 seconds
                     Op | INFO     | Program of Operation tt::operations::primary::Matmul                    finish

## Check that the output of the optimized version matches the output of the original implementation

In [12]:
torch_output = ttnn.to_torch(output)

optimized_output = ttnn.to_layout(optimized_output, ttnn.ROW_MAJOR_LAYOUT)
optimized_output = ttnn.from_device(optimized_output)
torch_optimized_output = ttnn.to_torch(optimized_output)

assert torch.allclose(torch_output, torch_optimized_output)

                     Op | INFO     | Program of Operation tt::tt_metal::Untilize                             finished in     0.393864225 seconds


## Close the device

In [13]:
ttnn.close(device)

                  Metal | INFO     | Closing device 0
